In [ ]:
pip install opencv-python mediapipe pandas numpy tensorflow keras

In [ ]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [ ]:
url = 'https://www.kaggle.com/datasets/debashishsau/aslamerican-sign-language-aplhabet-dataset'

# Download the data from Kaggle
!kaggle datasets download -d debashishsau/aslamerican-sign-language-aplhabet-dataset

Dataset URL: https://www.kaggle.com/datasets/debashishsau/aslamerican-sign-language-aplhabet-dataset
License(s): CC0-1.0
aslamerican-sign-language-aplhabet-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/aslamerican-sign-language-aplhabet-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2863).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2864).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2865).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2866).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2867).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2868).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2869).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (287).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2870).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2871).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2872).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2873).jpg  
  inflating: A

In [ ]:
import os
import cv2
import mediapipe as mp
import pandas as pd
from tqdm import tqdm

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to extract keypoints
def extract_keypoints(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    keypoints = []

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                keypoints.append(lm.x)
                keypoints.append(lm.y)
    else:
        keypoints = [None] * 42  # 21 keypoints * (x, y)

    return keypoints

# Define paths
DATASET_PATH = "/content/ASL_Alphabet_Dataset"
TRAIN_PATH = os.path.join(DATASET_PATH, "asl_alphabet_train")
TEST_PATH = os.path.join(DATASET_PATH, "asl_alphabet_test")

# Prepare DataFrame
def process_dataset(dataset_path, output_csv):
    data = []
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        if not os.path.isdir(label_path):
            continue

        for img_file in tqdm(os.listdir(label_path), desc=f"Processing {label}"):
            img_path = os.path.join(label_path, img_file)
            keypoints = extract_keypoints(img_path)
            data.append([label] + keypoints)

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False, header=False)
    print(f"CSV file saved: {output_csv}")

# Process train and test sets
process_dataset(TRAIN_PATH, "train_asl.csv")
process_dataset(TEST_PATH, "test_asl.csv")